In [1]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

import json
import os

with open("../config.json") as fptr:
    os.environ["OPENAI_API_KEY"] = json.load(fptr)["OPENAI_API_KEY"]

# setup network proxy
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

res = llm.invoke(messages)
print(res.content)

String theory is a theoretical framework in physics that aims to explain the fundamental nature of particles and their interactions by considering these particles as tiny, vibrating strings. It suggests that the fundamental building blocks of the universe are not point-like particles, as traditionally thought, but rather tiny, one-dimensional strings.

These strings can vibrate at different frequencies, and each frequency corresponds to a different particle with its own unique properties, such as mass and charge. In this way, string theory provides a more unified description of particles and their interactions compared to traditional particle physics theories, such as the Standard Model.

String theory also suggests the existence of extra dimensions beyond the three spatial dimensions that we experience in our everyday lives. These additional dimensions are compactified or curled up at incredibly small scales, making them difficult to detect directly. The idea of extra dimensions is a 

In [4]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, field_validator

class Joke(BaseModel):
    setup: str = Field(..., description="question to setup a joke")
    punchline: str = Field(..., description="answer to resolve the joke")

    @field_validator('setup')
    @classmethod
    def question_ends_with_question_mark(cls, field: str):
        if not field.endswith('?'):
            raise ValueError(f"Badly formed question: {field}")
        return field
    
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

joke_query = "Tell me a joke."
model_input  = prompt.format_prompt(query=joke_query)
model_output = llm.invoke(model_input.to_string())

parser.parse(model_output.content)

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')